In [7]:
import nglview as nv
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML
import os

In [48]:
def visualize_protein(up_acc):
    pdb_path = f"/data/alphafold/{up_acc[1:3]}/{up_acc}.pdb"  # AlphaFold PDB file with pLDDT in bfactor
    results_dir = f"data/results/{up_acc[0:2]}/{up_acc}/"

    view = nv.show_file(pdb_path)
    view._remote_call("setSize", target="Widget", args=["1200px", "800px"])  # Set width and height
    view.clear_representations()
    view.add_representation('cartoon', color='gray')  # Base representation

    # File paths
    discotope_path = f"{results_dir}{up_acc}_A_discotope3.csv"
    aggregation_path = f"{results_dir}{up_acc}_agg.txt"
    glycosylation_path = f"{results_dir}{up_acc}_glycosylation.csv"
    modifications_path = f"{results_dir}{up_acc}_modifications.csv"

    # Initialize data holders
    epitope_residues = []
    rsa_residues = []
    aggregation_residues = []
    glycosylation_residues = []
    modified_residues = []

    # Load the results files
        # Load data if files exist
    if os.path.exists(discotope_path):
        discotope_df = pd.read_csv(discotope_path)
        epitope_residues = discotope_df[discotope_df['epitope'] == True]['res_id'].tolist()
        rsa_residues = discotope_df[discotope_df['rsa'] > 0.5]['res_id'].tolist()

    if os.path.exists(aggregation_path):
        aggregation_df = pd.read_csv(aggregation_path, sep='\t')
        aggregation_residues = aggregation_df[aggregation_df['Aggregation'] > 50]['res'].tolist()

    if os.path.exists(glycosylation_path):
        glycosylation_df = pd.read_csv(glycosylation_path)
        glycosylation_residues = glycosylation_df['res'].tolist()

    if os.path.exists(modifications_path):
        modifications_df = pd.read_csv(modifications_path)
        modified_residues = modifications_df['res'].tolist()

    # Function to toggle visualizations
    def toggle_representation(visible_reps):
        view.clear_representations()
        view.add_representation('cartoon', color='gray')  # Base representation
        if 'epitope' in visible_reps:
            view.add_representation('cartoon', selection=' or '.join(map(str, epitope_residues)), color='red')
            view.add_representation('licorice', selection=' or '.join(map(str, epitope_residues)), color='red', opacity=0.5)
            view.add_representation('surface', selection=' or '.join(map(str, epitope_residues)), color='red', opacity=0)
        if 'aggregation' in visible_reps:
            view.add_representation('cartoon', selection=' or '.join(map(str, aggregation_residues)), color='orange')
            view.add_representation('licorice', selection=' or '.join(map(str, aggregation_residues)), color='orange', opacity=0.5)
            view.add_representation('surface', selection=' or '.join(map(str, aggregation_residues)), color='orange', opacity=0)
        if 'rsa' in visible_reps:
            view.add_representation('cartoon', selection=' or '.join(map(str, rsa_residues)), color='blue')
            view.add_representation('licorice', selection=' or '.join(map(str, rsa_residues)), color='blue', opacity=0.5)
            view.add_representation('surface', selection=' or '.join(map(str, rsa_residues)), color='blue', opacity=0)
        if 'plddt' in visible_reps:
            view.add_representation('cartoon', color_scheme='bfactor')
        if 'glycosylation' in visible_reps:
            view.add_representation('licorice', selection=' or '.join(map(str, glycosylation_residues)), color='cyan', opacity=0.5)
            view.add_representation('surface', selection=' or '.join(map(str, glycosylation_residues)), color='cyan', opacity=0)
        if 'modification' in visible_reps:
            view.add_representation('licorice', selection=' or '.join(map(str, modified_residues)), color='magenta', opacity=0.5)
            view.add_representation('surface', selection=' or '.join(map(str, modified_residues)), color='magenta', opacity=0)
            

    # Set for active toggles
    visible_reps = set()

    def create_toggle_button(label, key):
        button = widgets.ToggleButton(
            value=False,
            description=label,
            icon='eye'
        )
        
        def on_toggle(change):
            if change['new']:
                visible_reps.add(key)
            else:
                visible_reps.discard(key)
            toggle_representation(visible_reps)

        button.observe(on_toggle, names='value')
        return button

    def take_screenshot(_):
        screenshot_path = "screenshot.png"
        view.download_image(filename=screenshot_path)
        print(f"Screenshot saved to {screenshot_path}")

    screenshot_button = widgets.Button(description="Take Screenshot", icon="camera", button_style="info")
    screenshot_button.on_click(take_screenshot)

    # Create buttons
    epitope_button = create_toggle_button('Show Epitopes', 'epitope')
    aggregation_button = create_toggle_button('Show Aggregation', 'aggregation')
    rsa_button = create_toggle_button('Show RSA > 0.5', 'rsa')
    plddt_button = create_toggle_button('Show pLDDTs', 'plddt')
    glycosylation_button = create_toggle_button('Show Glycosylation', 'glycosylation')
    modification_button = create_toggle_button('Show Modifications', 'modification')

    

    display(widgets.HBox([epitope_button, aggregation_button, rsa_button, plddt_button, glycosylation_button, modification_button, screenshot_button]))
    display(view)




In [49]:
visualize_protein('Q15848')

NGLWidget()